<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_09_comparacion_con_regresion_logistica_y_evaluacion_de_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 09: Comparación con Regresión Logística y Evaluación de Modelos

## Objetivos
- Entrenar un modelo de regresión logística como baseline.
- Comparar métricas con XGBoost y CatBoost.
- Discutir implicaciones prácticas en banca.

---

## 1. Regresión Logística Baseline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier, plot_importance, DMatrix
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Verificar si catboost está instalado, si no instalarlo
try:
    from catboost import CatBoostClassifier
    #print("✅ CatBoost ya está instalado")
except ImportError:
    print("⚠️ CatBoost no está instalado. Instalando ahora...")
    !pip install -q catboost > /dev/null
    from catboost import CatBoostClassifier
    print("✅ CatBoost instalado correctamente")

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
df = pd.read_excel(url, header=1)

df.rename(columns={"default payment next month": "default"}, inplace=True)

In [ ]:
X = df.drop(columns=["ID","default"])
y = df["default"]

# Seleccionamos solo las columnas numéricas
numericas = X.select_dtypes(include=['int64', 'float64']).columns

# Inicializamos el scaler
scaler = StandardScaler()

# Ajustamos y transformamos
X_scaled = X.copy()
X_scaled[numericas] = scaler.fit_transform(X[numericas])

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

y_pred_lr = logreg.predict(X_test)
y_proba_lr = logreg.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred_lr))
print("AUC:", roc_auc_score(y_test, y_proba_lr))

---

## 2. Comparación de Modelos

In [ ]:
# Calcular el ratio
neg, pos = np.bincount(y_train)
scale = neg / pos

# Modelo con ajuste por desbalance y regularización
xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=4,
    random_state=42,
    eval_metric="logloss",
    scale_pos_weight=scale,   # 👈 desbalance
    reg_lambda=1.0,           # 👈 regularización L2 (default=1.0)
    reg_alpha=0.1             # 👈 regularización L1 (default=0.0, aquí lo activamos)
)

xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)
y_proba_xgb = xgb.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred_xgb))
print("AUC:", roc_auc_score(y_test, y_proba_xgb))

In [ ]:
cat = CatBoostClassifier(
    iterations=200,
    depth=6,
    learning_rate=0.1,
    random_seed=42,
    verbose=50,
    loss_function="Logloss",   # lo que se optimiza
    eval_metric="AUC",         # lo que se usa para seleccionar el "best iteration"
    auto_class_weights="Balanced",
    early_stopping_rounds=100,  # ⚡ se detiene si no mejora en 100 iteraciones
)

cat.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    #plot=True
)

y_pred_cat = cat.predict(X_test)
y_proba_cat = cat.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred_cat))
print("AUC:", roc_auc_score(y_test, y_proba_cat))

In [ ]:
results = {
    "Logistic Regression": roc_auc_score(y_test, y_proba_lr),
    "XGBoost": roc_auc_score(y_test, xgb.predict_proba(X_test)[:,1]),
    "CatBoost": roc_auc_score(y_test, cat.predict_proba(X_test)[:,1])
}

import pandas as pd
print(pd.DataFrame(results, index=["AUC"]).T)

---

## 3. Interpretación en Contexto Bancario

* **Regresión logística**: más interpretable, útil para auditoría/regulación.
* **XGBoost / CatBoost**: mejor performance, útil en scoring de riesgo real.
* Elección depende del trade-off entre **precisión vs explicabilidad**.

---

## 4. Visualización de Importancia de Features

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))
ax2 = ax1.twinx()

idx = np.arange(len(feature_names))

ax1.bar(idx - 0.2, xgb_feat, width=0.4, label="XGBoost (0–1)", alpha=0.7)
ax2.bar(idx + 0.2, cat_feat, width=0.4, label="CatBoost (0–100)", alpha=0.5, color="tab:orange")

ax1.set_xticks(idx)
ax1.set_xticklabels(feature_names, rotation=45, ha="right")
ax1.set_ylabel("XGBoost (suma=1)")
ax2.set_ylabel("CatBoost (suma≈100)")
plt.title("Importancia de Variables")
fig.tight_layout()
fig.legend(loc="upper right")
plt.show()


## 5. Preguntas de Discusión

### 1. ¿Qué métrica es más relevante para un banco: AUC, Recall o Precision?

* **AUC (Area Under the Curve)** mide la capacidad global del modelo para distinguir entre clases (por ejemplo, buenos vs. malos pagadores). Es útil para comparar modelos, pero no dice directamente el costo de errores.
* **Recall (Sensibilidad o Tasa de Verdaderos Positivos)** mide cuántos malos pagadores fueron correctamente identificados. Es muy crítico en banca, porque perder un cliente de alto riesgo (falso negativo) significa pérdidas directas.
* **Precision (Precisión o Valor Predictivo Positivo)** mide cuántos de los clientes clasificados como “malos pagadores” realmente lo son. En banca, baja precisión implica que estamos rechazando muchos clientes que en realidad pagarían (pérdida de negocio).

👉 **Relevancia:**

* Para **riesgo crediticio**, normalmente se prefiere **Recall** alto, porque lo más costoso es aprobar a alguien que no va a pagar (falso negativo).
* Sin embargo, el banco también debe cuidar la **Precision**, porque si rechaza demasiados buenos clientes pierde mercado.
* El **AUC** sirve como comparador general de modelos, pero la decisión de negocio suele ponderar Recall y Precision en función de la estrategia del banco.

---

### 2. ¿Por qué un banco podría preferir regresión logística aunque rinda peor?

* **Interpretabilidad:** la regresión logística es lineal y sus coeficientes se interpretan como odds ratios, lo que permite explicar fácilmente por qué un cliente fue aprobado o rechazado. Esto es crítico por **regulación** y para la confianza del negocio.
* **Simplicidad:** es más fácil de mantener, auditar y comunicar a stakeholders (reguladores, comité de riesgos, auditores internos).
* **Robustez:** la regresión logística es menos propensa a sobreajustar que modelos más complejos (XGBoost, redes neuronales) si los datos no son muy grandes.
* **Costos regulatorios:** en muchos países, los reguladores exigen modelos explicables para decisiones crediticias; un “black box” puede ser problemático.

👉 Aunque XGBoost tenga mejor AUC, el banco podría preferir regresión logística porque **sacrifica algo de performance a cambio de transparencia, auditabilidad y cumplimiento regulatorio**.

---

### 3. ¿Cómo monitorearías el drift de estos modelos en producción?

Primero: **¿Qué es drift?**
El **drift** ocurre cuando la distribución de los datos de producción se desvía respecto de los datos usados en el entrenamiento. Esto degrada la performance del modelo con el tiempo. Hay varios tipos:

* **Data Drift (o covariate shift):** cambian las distribuciones de las variables de entrada (ejemplo: cambia la proporción de clientes jóvenes vs. mayores que piden crédito).
* **Concept Drift:** cambia la relación entre variables y la etiqueta (ejemplo: antes, cierto nivel de deuda era señal de riesgo, pero en crisis económica deja de serlo).
* **Label Drift:** cambian las proporciones de la variable objetivo (ejemplo: aumenta la tasa real de impagos en la economía).

👉 **Cómo monitorear drift en producción:**

1. **Comparar distribuciones de features:** usar tests estadísticos (Kolmogorov–Smirnov, Jensen-Shannon, PSI – Population Stability Index) para detectar si las variables de entrada cambiaron.
2. **Monitoreo de métricas de performance:** seguir AUC, Recall, Precision en muestras de datos con etiquetas recientes. Si bajan, puede haber concept drift.
3. **Alertas tempranas:** implementar dashboards (ej. con EvidentlyAI, Fiddler, MLflow) que automaticen la detección de drift y generen alertas.
4. **Retraining periódico:** si el drift persiste, reentrenar con datos recientes para que el modelo se adapte.

👉 En banca, el drift es **crítico** porque los hábitos de pago de los clientes cambian con la economía, nuevas regulaciones o crisis financieras.

---

✅ Resumen:

* **Métrica clave:** Recall (pero sin olvidar Precision).
* **Modelo simple:** regresión logística se prefiere por interpretabilidad y regulación.
* **Drift:** desviación en datos o relaciones; se monitorea con tests de distribución, performance en producción y retraining periódico.
